# Downloading data from Close CRM 

In [54]:
import json
import pandas as pd
from closeio_api import Client
from google.cloud import bigquery
import datetime
import pandas
import pytz
import os
import pyarrow
import requests
from pandas import json_normalize

In [2]:
class CloseApiWrapper(Client):
    """
    Close API wrapper that makes it easier to paginate through resources and get all items
    with a single function call alongside some convenience functions (e.g. getting all lead statuses).
    """

    def __init__(
        self, api_key=None, tz_offset=None, max_retries=5, development=False
    ):
        super().__init__(
            api_key=api_key,
            tz_offset=tz_offset,
            max_retries=max_retries,
            development=development,
        )

    def get_lead_statuses(self):
        organization_id = self.get('me')['organizations'][0]['id']
        return self.get(
            f"organization/{organization_id}",
            params={"_fields": "lead_statuses"},
        )["lead_statuses"]

    def get_opportunity_pipelines(self):
        organization_id = self.get('me')['organizations'][0]['id']
        return self.get(
            f"organization/{organization_id}",
            params={"_fields": "pipelines"},
        )["pipelines"]

    def get_custom_fields(self, type):
        return self.get(f"custom_field_schema/{type}")["fields"]

    def get_opportunity_statuses(self):
        organization_id = self.get('me')['organizations'][0]['id']
        pipelines = self.get(
            f"organization/{organization_id}",
            params={"_fields": "pipelines"},
        )["pipelines"]

        opportunity_statuses = []
        for pipeline in pipelines:
            opportunity_statuses.extend(pipeline['statuses'])

        return opportunity_statuses

    def get_all_items(self, url, params=None):
        if params is None:
            params = {}

        items = []
        has_more = True
        offset = 0
        while has_more:
            params["_skip"] = offset
            resp = self.get(url, params=params)
            items.extend(resp['data'])
            offset += len(resp["data"])
            has_more = resp["has_more"]

        return items

In [3]:
filtro='''{
    "limit": null,
    "query": {
        "negate": false,
        "queries": [
            {
                "negate": false,
                "object_type": "lead",
                "type": "object_type"
            },
            {
                "negate": false,
                "queries": [
                    {
                        "negate": false,
                        "queries": [
                            {
                                "condition": {
                                    "type": "term",
                                    "values": [
                                        "Agora",
                                        "Cover Manager",
                                        "Dish - Makro",
                                        "El Tenedor",
                                        "Five stars restaurant reservation",
                                        "Hospitality Digital",
                                        "None",
                                        "Resmio",
                                        "Restoo",
                                        "Sevenrooms",
                                        "Spotlinker",
                                        "Superb"
                                    ]
                                },
                                "field": {
                                    "custom_field_id": "cf_0pYIi4FHHnhOM1ifbXJYZLcfDU13bEH9eHYslzr1Zab",
                                    "type": "custom_field"
                                },
                                "negate": false,
                                "type": "field_condition"
                            },
                            {
                                "condition": {
                                    "object_ids": [
                                        "stat_8t23pHs9IQotTZunnUeWB6w6RD2Na1Dob5fk9eAsLUd",
                                        "stat_9xQf6cjuUwuQ92zPUmUnakvs5YhjUDtvkQBzenguCET",
                                        "stat_Gb5oaZjacOsnVKwNZFdDODIsSTipR1HTAFwBfWPziKG",
                                        "stat_L7NEk2Kc2pX7WMsBQn07Vs3KTwG3lkp70p0446VZsTY",
                                        "stat_MD0TdHBJkRjxaCfemqLSTgwmLC7FQeaxt7Po9R1Ng04",
                                        "stat_NeP2VGE2PASFs95QHEnxEhXi382RFX2fDPjCK29If1r",
                                        "stat_PSdJsfSHu9Wf73moypWSQpNITASJ3W3Tzi0VU88Nsjy",
                                        "stat_XdRA89GRcYWzd4q14iCXIEiue0Po6Us4G68b2t8GT4v",
                                        "stat_cfQPZ3aedcrxwaDUrK37oXzIRwMUKsODlSBJWLYlyHh",
                                        "stat_hWNOcRMXFrdlQECF0itYxqxzYs2qmR6OeQepEWhu1YM",
                                        "stat_iEgcVYfhfypOoqZ87h08TLBmpAplUPFpGNYut2bESGR",
                                        "stat_wnopZFSHmC9PCVDHgym1wrpEUNT0XQzF1TJ8Uu3WIlN"
                                    ],
                                    "reference_type": "status.lead",
                                    "type": "reference"
                                },
                                "field": {
                                    "field_name": "status_id",
                                    "object_type": "lead",
                                    "type": "regular_field"
                                },
                                "negate": false,
                                "type": "field_condition"
                            },
                            {
                                "condition": {
                                    "type": "exists"
                                },
                                "field": {
                                    "field_name": "name",
                                    "object_type": "lead",
                                    "type": "regular_field"
                                },
                                "negate": false,
                                "type": "field_condition"
                            }
                        ],
                        "type": "and"
                    }
                ],
                "type": "and"
            }
        ],
        "type": "and"
    },
    "results_limit": null,
    "sort": []
}
'''

filtro= json.loads(filtro) #cargar el json y lo convierte en diccionario de python
filtro

{'limit': None,
 'query': {'negate': False,
  'queries': [{'negate': False, 'object_type': 'lead', 'type': 'object_type'},
   {'negate': False,
    'queries': [{'negate': False,
      'queries': [{'condition': {'type': 'term',
         'values': ['Agora',
          'Cover Manager',
          'Dish - Makro',
          'El Tenedor',
          'Five stars restaurant reservation',
          'Hospitality Digital',
          'None',
          'Resmio',
          'Restoo',
          'Sevenrooms',
          'Spotlinker',
          'Superb']},
        'field': {'custom_field_id': 'cf_0pYIi4FHHnhOM1ifbXJYZLcfDU13bEH9eHYslzr1Zab',
         'type': 'custom_field'},
        'negate': False,
        'type': 'field_condition'},
       {'condition': {'object_ids': ['stat_8t23pHs9IQotTZunnUeWB6w6RD2Na1Dob5fk9eAsLUd',
          'stat_9xQf6cjuUwuQ92zPUmUnakvs5YhjUDtvkQBzenguCET',
          'stat_Gb5oaZjacOsnVKwNZFdDODIsSTipR1HTAFwBfWPziKG',
          'stat_L7NEk2Kc2pX7WMsBQn07Vs3KTwG3lkp70p0446VZsTY',
  

In [195]:
params = {}
has_more = True
offset = 0
calls = []
display_names = {}
api = CloseApiWrapper("api_3yFWGD0paiQ2UKvsQvrahS.6ALHM6ZnMvbnfv0jTpjA7Y")
resp=api.get( 
        'lead',
        params={
                '_skip':offset,
                'query': filtro,
                '_fields': 'id,display_name,status_label'
            },)

resp2=resp.get("data") ##esto es una lista
df=pd.DataFrame()

for i in resp2:
    df=pd.DataFrame(pd.concat([df,pd.DataFrame(i.values(), index=i.keys()).transpose().set_index("id")]))

print(df)
#df.to_csv(r'C:\Users\jaime\Downloads\Bookline\prueba.csv')








## Subir datos a query

In [31]:
pip install -- pyarrow

In [15]:
pip install --upgrade google-cloud-bigquery

Note: you may need to restart the kernel to use updated packages.


In [201]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/jaime/Downloads/Bookline/looker-studio-377715-075aecd69bce.json"
client = bigquery.Client()
table_id= "looker-studio-377715.restaurantes" #esto lo he sacado de big query
dataframe=df


##Aqui ya hacer el query
## se puede tmb hacer un index 

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        
        bigquery.SchemaField("id", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("display_name", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("status_label", bigquery.enums.SqlTypeNames.STRING),
    
        
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    dataframe, table_id, job_config=job_config
) 


 
#tmb me da error de table id primero q no esta bien hecho, lo cambio y me dice que no esta en formato SQL. Y al ponerlo en formato SQL me da error 





job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

NotFound: 404 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/looker-studio-377715/jobs?uploadType=multipart: Not found: Dataset looker-studio-377715:looker-studio-377715